<table width=100%>
    <tr>
        <td width = 40% align = "left">
            <h3> MTH786 Machine Learning with Python</h3>
        </td>
        <td width = 35%>            
        </td>
        <td width = 25% align = "left">
            <h3>Semester A </h3>
        </td>
    </tr>
    <tr>
        <td width = 40% align = "left">
            <h3> Coursework 10</h3>
        </td>
        <td width = 35%>            
        </td>
        <td width = 25% align = "left">
            <h3>Dr Nicola Perra </h3>
        </td>
    </tr>
</table>

In [1]:
import numpy as np
from numpy.testing import assert_array_almost_equal, assert_array_equal
import matplotlib.pyplot as plt
%matplotlib inline

### Multinomial logistic classification
We now move on to multinomial logistic regression for multi-class classfication problems. To this end we will use a synthetic dataset built for these types of taks. The dataset consits on s samples, 10 features and different categories


We start by loading data. **Important:** please check that file $\mathtt{dataset.csv}$ is located in the same folder with your Jupyter notebook.

In [2]:
# first step is to get the data, labels and inputs
complete_data = np.genfromtxt("dataset.csv", skip_header = 1, delimiter = ',')
labels = complete_data[:, -1].astype(int) - np.min(complete_data[:, -1].astype(int))
inputs = complete_data[:, :-1]

1. Implement function $\mathtt{linear\_regression\_data}$ that computes (and outputs) the linear regression data matrix. The function should take the NumPy array _data_inputs_ as argument.

In [3]:
def linear_regression_data(data_inputs):
    first_column = np.ones((len(data_inputs), 1))
    X_matrix = np.c_[first_column,data_inputs]
    return X_matrix

2. Write two functions 
- $\mathtt{standardise}$ to standardise the columns of a multi-dimensional array. The function $\mathtt{standardise}$	takes the multi-dimensional array _data_matrix_ as its input argument. It subtracts the means from each column and divides by the standard deviations. It returns the _standardised_matrix_, the _row_of_means_ and the _row_of_standard_deviations_.
- $\mathtt{de\_standardise}$ to de-standardise the columns of a multi-dimensional array. The function $\mathtt{de\_standardise}$ reverses the above operation. It takes a _standardised_matrix_, the _row_of_means_ and the _row_of_standard_deviations_ as its arguments and returns a matrix for which the standardisation process is reversed.

In [4]:
def standardise(data_matrix):
    row_of_means = np.mean(data_matrix, axis=0)
    standardised_matrix = data_matrix - row_of_means
    row_of_stds = np.std(standardised_matrix, axis=0)
    return (standardised_matrix / row_of_stds), row_of_means, row_of_stds

In [5]:
def de_standardise(standardised_matrix, row_of_means, row_of_stds):
    matrix = np.copy(standardised_matrix * row_of_stds)
    return matrix + row_of_means

Now we standardise the input data and prepare data matrix for a multinomial logistic regression. 

In [6]:
data_inputs, row_of_means, row_of_stds = standardise(inputs)
data_matrix = linear_regression_data(data_inputs)

3. As a first step, implement the softmax function $\mathtt{softmax\_function}$ as defined in the lectures. The function takes the NumPy array _argument_ as its main argument, but also has an optional _axis_ argument to determine across which array-dimension you apply the softmax operation (axis = 0 for columns, axis = 1 for rows, ...). If this argument is not specified (or set to _None_), then the softmax operation is applied to the entire array. Make sure your function works at least for NumPy arrays _argument_ with arbitrary numerical values and dimension one or two.

In [8]:
def softmax_function(argument, axis=None):    
    if axis == None:
        output = np.exp(argument - np.max(argument))
        output = output / np.sum(output)
    else:
        output = np.exp(argument - np.expand_dims(np.max(argument, axis), axis))
        output = output / np.expand_dims(np.sum(output, axis), axis)
    return output

Test your $\mathtt{softmax\_function}$ function with the following cell. 

In [9]:
assert_array_almost_equal(softmax_function(np.array([[1.5], [0.3], [-3.7]])), \
                          np.array([[0.76528029], [0.23049799], [0.00422172]]))

In [10]:
assert_array_almost_equal(softmax_function(np.array([[1.5, 3], [0.3, -0.7], [-3.7, 2]]), axis=0), \
                          np.array([[0.76528029, 0.71807976], [0.23049799, 0.01775346], \
                                    [0.00422172, 0.26416678]]))

In [11]:
assert_array_almost_equal(softmax_function(np.array([[1.5, 3], [0.3, -0.7], [-3.7, 2]]), axis=1), \
                          np.array([[0.182426, 0.817574],\
                                    [0.731059, 0.268941],\
                                    [0.003335, 0.996665]]))

4. Implement a function $\mathtt{model\_function}$ that outputs the values of the linear model function. Unlike the case of binary classification, the $\mathtt{model\_function}$ should return a vector of dimension $K$ for each data sample. This vector is defined as
$$
f\left(\mathbf{x},\mathbf{W}\right) = 
\left( \left\langle \phi\left(\mathbf{x}\right), \mathbf{w}^{(1)}\right\rangle \quad \left\langle \phi\left(\mathbf{x}\right), \mathbf{w}^{(2)}\right\rangle \quad \ldots \quad \left\langle \phi\left(\mathbf{x}\right), \mathbf{w}^{(K)}\right\rangle\right),
$$
where 
$$
\mathbf{W} = 
\begin{pmatrix}
\vdots & \vdots & \ldots & \vdots \\
\mathbf{w}^{(1)} & \mathbf{w}^{(2)} & \ldots & \mathbf{w}^{(K)}\\
\vdots & \vdots & \ldots & \vdots \\
\end{pmatrix}
$$
is a mathematical representation of the weights matrix.

As in the binary classification case, the arguments of $\mathtt{model\_function}$ are the data matrix _data_matrix_ and weights that are now named _weights_matrix_.

In [12]:
def model_function(data_matrix, weight_matrix):
    return data_matrix @ weight_matrix

Test your $\mathtt{model\_function}$ function with the following cell. 

In [13]:
test_data_matrix = np.array([[1,2,3],[1,4,5],[1,6,7]])
test_weight_matrix = np.array([[-1,1],[0,1],[1,0]])
assert_array_almost_equal(model_function(test_data_matrix, test_weight_matrix),np.array([[2,3],[4,5],[6,7]]))

5. Write a function $\mathtt{multinomial\_prediction\_function}$ that turns your model function into labels. The function takes the arguments _data_matrix_ and _weight_matrix_ as inputs and returns a vector of labels with values in $\{0, K - 1 \}$ as its output.

In [14]:
def multinomial_prediction_function(data_matrix, weight_matrix):    
    return np.argmax(model_function(data_matrix, weight_matrix), axis=1)

Test your $\mathtt{multinomial\_prediction\_function}$ function with the following cell.

In [15]:
test_data_matrix = np.array([[6, 4, 5], [1, 2, 8], [-3, 3, 6], [6, 5, -100], [5, 7, 2]])
test_weight_matrix = np.array([[2,  1, -2, -4], [ 2, -5,  1,  4], [-2, -3, -1, -2]])
assert_array_almost_equal(multinomial_prediction_function(test_data_matrix, test_weight_matrix),np.array([0,2,3,1,0]))

6. Implement a function $\mathtt{gradient\_descent}$ that performs gradient descent to numerically approximate a minimiser of a convex function. The function should take the following arguments
- *objective* - a lambda-function representing function $E$. This itself should take a NumPy array as its argument and return a real number.
- *gradient* - a lambda-function representing function $\nabla E$. This itself should take a NumPy array as its argument and return a NumPy array representation of the gradient $\nabla E$.
- *initial_ weights* - a NumPy array with initial values $\mathbf{w}^{(0)}$ for the first iterate 
- *step_size* - a step-size parameter $\tau$ for the gradient descent step
- *no_of_iterations* - an integer parameter that controls the number of iterations
- *print_output* - an integer parameter that controls how often you are printing an intermediate result. If say *print_output = 100*, then after every 100th iteration you are asked to print your current iterate and a value of the objective as *Iteration k/m, objective = o.*, where $k$ is a number of current iteration, $m$ is a total number of iterations, and $o$ is a value of the objective evaluated at current iterate.

Implement the function so that it returns a NumPy array of the weights obtained after gradient descent together with a list of objective values for all iterates.

In [16]:
def gradient_descent(objective,
                     gradient,
                     initial_weights,
                     step_size=1,
                     no_of_iterations=100,
                     print_output=10):
    
    objective_values = []
    weights = np.copy(initial_weights)
    objective_values.append(objective(weights))
    for counter in range(no_of_iterations):
        weights -= step_size * gradient(weights)
        objective_values.append(objective(weights))
        if (counter + 1) % print_output == 0:
            print("Iteration {k}/{m}, objective = {o}.".format(k=counter+1, 
                                                               m=no_of_iterations, o=objective_values[counter]))
    print("Iteration completed after {k}/{m}, objective = {o}.".format(k=counter+ 1, 
                                                                        m=no_of_iterations,
                                                                       o=objective_values[counter]))
    return weights, objective_values

Test your function with the following unit test

In [17]:
test_matrix_m = np.array([[3, 1], [2, 4]])
test_vector_v = np.array([5, 6])
# usual energy function
test_objective = lambda x: x.T @ (test_matrix_m @ x) + x @ test_vector_v
# we know the expression for the gradient
test_gradient = lambda x: (test_matrix_m + test_matrix_m.T) @ x + test_vector_v

test_initial_weights = np.array([0.0, 0.0])
test_step_size = 0.9 / (np.linalg.norm(test_matrix_m + test_matrix_m.T))
test_no_of_iterations = 100
test_print_output = 10
assert_array_almost_equal(gradient_descent(test_objective, test_gradient, \
                                           test_initial_weights,test_step_size,\
                                           test_no_of_iterations, test_print_output)[0],np.array([-0.564103, -0.538462]))

Iteration 10/100, objective = -3.0256011293288547.
Iteration 20/100, objective = -3.0256410067129824.
Iteration 30/100, objective = -3.025641025632046.
Iteration 40/100, objective = -3.0256410256410207.
Iteration 50/100, objective = -3.025641025641026.
Iteration 60/100, objective = -3.0256410256410255.
Iteration 70/100, objective = -3.0256410256410264.
Iteration 80/100, objective = -3.0256410256410255.
Iteration 90/100, objective = -3.0256410256410255.
Iteration 100/100, objective = -3.0256410256410255.
Iteration completed after 100/100, objective = -3.0256410256410255.


7. Next, write a function $\mathtt{one\_hot\_vector\_encoding}$ that converts an NumPy array _labels_ with values in the range of $\{0, K - 1\}$ into so-called one-hot vector encodings. For example, for $K = 3$ and a label vector $\text{labels} = 
\begin{pmatrix} 2 & 0 & 1 & 2\end{pmatrix}^\top$, the output of $\mathtt{one\_hot\_vector\_encoding}$ should be a two-dimensional NumPy array of the form
$$
\begin{pmatrix} 0 & 0 & 1 \\ 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 
\end{pmatrix}.
$$

Note how we have 1 in each row correspondent to the value of the label

In [18]:
def one_hot_vector_encoding(labels):        
    no_of_classes = np.max(labels) + 1
    output = np.zeros((len(labels), no_of_classes)) # as many rows as data points, as many columns as classes
    output[np.arange(len(labels)), labels] = 1
    # why it works?
    # np.arange(x) creates an array from 0 to x-1
    # labels is an array that tells us the id of each row that should be one!
    return output

In [19]:
# this is equivalent to
def one_hot_vector_encoding_2(labels):        
    no_of_classes = np.max(labels) + 1
    output = np.zeros((len(labels), no_of_classes))
    counter=0
    for i in labels:
        output[counter,i]=1
        counter+=1
    return output

Test your $\mathtt{one\_hot\_vector\_encoding}$ function with the following cell

In [20]:
# let's try it 
labels=np.array([1, 2, 0, 3])
print (one_hot_vector_encoding(labels))
print ('-------------------------')
print (one_hot_vector_encoding_2(labels))

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
-------------------------
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


In [21]:
# let's try another one
labels=np.array([1,0,1,0]) # since we have two classes we have just two columns
print (one_hot_vector_encoding(labels))

[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


8. Implement the cost function and gradient for the multinomial logistic regression in terms of two functions $\mathtt{multinomial\_logistic\_regression\_cost\_function}$ and $\mathtt{multinomial\_logistic\_regression\_gradient}$. As in the binary classification case, the arguments are the data matrix _data_matrix_ and weights that are now named _weights_matrix_. Instead of passing on labels as _outputs_ as in the binary case, you pass the one hot vector encoding representation _one_hot_vector_encodings_ as your third argument. Return the cost function value, respectively the gradient, following the mathematical formulas in the lecture notes.

$$
L\left(\mathbf{W}\right) = \sum\limits_{i=1}^s\log\left[\sum\limits_{k=1}^K \mathrm{e}^{f\left(\mathbf{x}^{(i)},\mathbf{W}\right)_k}\right]
- \sum\limits_{i=1}^s\sum\limits_{k=1}^K \mathbf{1}_{y_i=k}f\left(\mathbf{x}^{(i)},\mathbf{W}\right)_k,
$$
where $f\left(\mathbf{x}, \mathbf{W}\right)$ is a model function. In the case of linear model function 
$$
f\left(\mathbf{x},\mathbf{W}\right) = 
\left( \left\langle \phi\left(\mathbf{x}\right), \mathbf{w}^{(1)}\right\rangle \quad \left\langle \phi\left(\mathbf{x}\right), \mathbf{w}^{(2)}\right\rangle \quad \ldots \quad \left\langle \phi\left(\mathbf{x}\right), \mathbf{w}^{(K)}\right\rangle\right)
$$
one can write
$$
\begin{align*}
L\left(\mathbf{W}\right) &=& \sum\limits_{i=1}^s\log\left[\sum\limits_{k=1}^K \mathrm{e}^{\left\langle \phi\left(\mathbf{x}^{(i)}\right), \mathbf{w}^{(k)}\right\rangle}\right]
- \sum\limits_{i=1}^s\sum\limits_{k=1}^K \mathbf{1}_{y_i=k}\left\langle \phi\left(\mathbf{x}^{(i)}\right), \mathbf{w}^{(k)}\right\rangle
\\
\frac{\partial L\left(\mathbf{W}\right)}{\partial \mathbf{w}^{(p)}_q} &=& \sum\limits_{i=1}^s \phi\left(\mathbf{x}^{(i)}\right)_q\mathrm{softmax}\left(f\left(\mathbf{x}^{(i)}, \mathbf{W}\right)\right)_p
- \sum\limits_{i=1}^s\mathbf{1}_{y_i = p} \phi\left(\mathbf{x}^{(i)}\right)_q.
\end{align*}
$$

In [20]:
def multinomial_logistic_regression_cost_function(data_matrix, weight_matrix, one_hot_vector_encodings):    
    model_evaluation = model_function(data_matrix, weight_matrix)
    return np.sum(np.log(np.sum(np.exp(model_evaluation), axis=1)) - np.sum(one_hot_vector_encodings * model_evaluation, axis=1))

In [21]:
def multinomial_logistic_regression_gradient(data_matrix, weight_matrix, one_hot_vector_encodings):    
    model_evaluation = model_function(data_matrix, weight_matrix)
    return data_matrix.T @ (softmax_function(model_evaluation, axis=1) -one_hot_vector_encodings)

Test your functions with the following cells

In [22]:
test_data_matrix = np.array([[6, 4, 5], [1, 2, 8], [-3, 3, 6], [6, 5, -100], [5, 7, 2]])
test_weight_matrix = np.array([[2,  1, -2, -4], [ 2, -5,  1,  4], [-2, -3, -1, -2]])


test_one_hot_vector_encoding = np.array([[1., 0., 0., 0.], [0., 0., 1., 0.], \
                                         [0., 0., 0., 1.], [0., 1., 0., 0.], [1., 0., 0., 0.]])
assert_array_almost_equal(multinomial_logistic_regression_cost_function(test_data_matrix, \
                                                                        test_weight_matrix, \
                                                                        test_one_hot_vector_encoding),\
                         0.1430551433917744)

In [23]:
test_data_matrix = np.array([[6, 4, 5], [1, 2, 8], [-3, 3, 6], [6, 5, -100], [5, 7, 2]])
test_weight_matrix = np.array([[2,  1, -2, -4], [ 2, -5,  1,  4], [-2, -3, -1, -2]])
test_one_hot_vector_encoding = np.array([[1., 0., 0., 0.], [0., 0., 1., 0.], \
                                         [0., 0., 0., 1.], [0., 1., 0., 0.], [1., 0., 0., 0.]])
assert_array_almost_equal(multinomial_logistic_regression_gradient(test_data_matrix, \
                                                                   test_weight_matrix, \
                                                                   test_one_hot_vector_encoding),\
                         np.array([[ 1.173099e-01,  1.203832e-11, -1.335569e-01,  1.624699e-02],\
                                   [ 2.346201e-01,  2.407656e-11, -2.660032e-01,  3.138308e-02],\
                                   [ 9.384832e-01,  9.630610e-11, -1.064753e+00,  1.262698e-01]]))

9. Write a function $\mathtt{classification\_accuracy}$ that takes two NumPy array arguments $\mathtt{predicted\_labels}$ and $\mathtt{true\_labels}$ and evaluates the ratio of labels that coincide in both arrays to the number of all labels.

In [24]:
def classification_accuracy(estimated_labels, true_labels):
    equal_labels = estimated_labels == true_labels
    return np.mean(equal_labels)

Test your function with the following unit tests.

In [40]:
test_estimated_labels = np.array([0,4,0,0,2,4,0,0,2,2,2,3,3,1,3,4,0,3,4,0,1,1,2,0,0,0,\
                                  1,4,4,4,3,0,4,2,4,4,4,2,2,1,4,3,2,2,1,1,4,3,3,0,4,3,\
                                  0,0,0,2,4,3,4,3,1,3,2,4,2,3,2,3,2,3,1,0,4,3,2,3,1,3,\
                                  4,1,3,1,4,0,4,4,1,2,3,1,1,4,3,1,3,0,2,0,0,1])
test_true_labels = np.array([4,4,4,4,1,3,0,4,1,0,2,3,1,2,0,1,3,4,3,4,4,4,2,0,3,4,3,2,3,\
                             0,1,0,4,3,2,2,1,2,2,1,4,0,4,4,3,0,0,0,3,2,3,0,0,3,3,4,2,2,\
                             3,4,2,3,3,0,0,2,0,3,0,4,4,4,4,1,2,3,1,3,1,1,2,0,2,3,3,3,4,\
                             0,0,3,1,0,3,0,3,1,0,0,4,4])
assert_array_almost_equal(
    classification_accuracy(test_estimated_labels, test_true_labels), 0.26)

10. Finally, we would like to apply all the above to the data. I.e.,
- define two lambda functions called $\mathtt{objective}$, $\mathtt{gradient}$ that take $1$ argument _weight_;
- define _initial_weights_ to be a zero matrix of appropriate size;
- let the step size parameter _step_size_ be equal to $3.9/\left\|\mathbf{X}\right\|^2$;
- run 10,000 iterations of the gradient descent procedure to find an optimal weights _optimal_weights_;
- evaluate an accuracy rate _accuracy_rate_ evaluated for the model defined to have _optimal_weights_ weights.

In [25]:
# let us get back the data 

complete_data = np.genfromtxt("dataset.csv", skip_header = 1, delimiter = ',')
labels = complete_data[:, -1].astype(int) - np.min(complete_data[:, -1].astype(int))
inputs = complete_data[:, :-1]

#don't forget to standardise it
data_inputs, row_of_means, row_of_stds = standardise(inputs)
data_matrix = linear_regression_data(inputs)

# we need the OHV
OHV = one_hot_vector_encoding(labels)

# then as usual the objective function and it gradient
objective = lambda weights: multinomial_logistic_regression_cost_function(data_matrix, weights,OHV)
gradient = lambda weights: multinomial_logistic_regression_gradient(data_matrix, weights, OHV)

step_size = 3.9/(np.linalg.norm(data_matrix))**2

initial_weight_matrix = np.zeros((data_matrix.shape[1], OHV.shape[1]))
optimal_weights, objective_values = gradient_descent(objective, gradient,
                                                               initial_weight_matrix, step_size, 
                                                               no_of_iterations=10000, print_output=1000)

accuracy_rate = classification_accuracy(multinomial_prediction_function(data_matrix,
                                                                             optimal_weights),labels)

Iteration 1000/10000, objective = 1439.189196039968.
Iteration 2000/10000, objective = 1433.3815618034994.
Iteration 3000/10000, objective = 1431.1092776592886.
Iteration 4000/10000, objective = 1429.9411000503446.
Iteration 5000/10000, objective = 1429.254721096464.
Iteration 6000/10000, objective = 1428.8165528337333.
Iteration 7000/10000, objective = 1428.5204216070545.
Iteration 8000/10000, objective = 1428.3116885725212.
Iteration 9000/10000, objective = 1428.1596600836444.
Iteration 10000/10000, objective = 1428.045946444532.
Iteration completed after 10000/10000, objective = 1428.045946444532.


In [44]:
print("The mulnomial logistic regression successfully classified {acc:2.2f} % of data".format(acc = 100*accuracy_rate))

The mulnomial logistic regression successfully classified 60.94 % of data


### Two layers Neural Network for mutliclass classification

In the second part of the assignment you are asked to try the same classification task but considering a neural network with $L = 2$ layers. All layers have the same activation function which is given by the $\mathtt{softmax\_function}$ and same type of model function given by an affine-linear transformation. Namely, we define
$$
\left\{
\begin{array}{ll}
Z^{(\ell)} &=  \widetilde{X}^{(\ell-1)}W^{(\ell)},\\
X^{(\ell)} &=  \mathrm{softmax}\left(Z^{(\ell)}\right)
\end{array}
\right.,
\label{eq:nn_model}
$$
for $\ell=1,\ldots,L$, where 
- $X^{(\ell)}$ is a mathematical representation of data _inputs_ at the layer $\ell+1$, in particular $X^{(0)}$ is a mathematical representation of an input at the first layer, which just represents data samples;
- $Z^{(\ell)}$ is a mathematical representation of model function values at the layer $\ell$, in particular $Z^{(L)}$ is a mathematical representation the final output of neural network;
- for every matrix $M$ we write $\widetilde{M}$ to denote an augmented matrix $M$, i.e. the one with artificial column of ones inserted at the beginning of a matrix; 
- $W^{(\ell)}$ are weight matrices representing linear transformation applied at the layer $\ell$ for $\ell = \overline{1,L}$ that have dimensions $\left( d_{\ell} + 1 \right)\times d_{\ell+1}$ with $d_1 = d$, $d_{L+1} = K$, while $d_2,\ldots,d_L$ can take any integer values. In this exercise we would assume $d_2 = \ldots = d_L = d$.

By training a neural network we understand identifying optimal values of parameters $\mathbf{W} = \left(\mathbf{W}^{(1)}, \mathbf{W}^{(2)}, \ldots, \mathbf{W}^{(L)}\right)$ such that the cost function
$$
L\left(\mathbf{W}\right) = 
\sum\limits_{i=1}^s \log\left(\sum\limits_{j=1}^K
\mathrm{e}^{Z^{(L)}_{i,j}}\right)
-
\sum\limits_{i=1}^s\sum\limits_{j=1}^K
\mathbf{1}_{y_i = j} Z^{(L)}_{i,j}.
$$
is minimised.

The optimisation problem is set to be solved by using the gradient descent method. This involves a backpropagation approach to a calculation of the gradient $\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d}\mathbf{W}}$ (see lecture notes). To help you with the problem, we provide you with a full set of equations you need to use to solve the problem. Every derivative here is thought as a gradient and is written in the form of a corresponding size matrix. For example,
the derivative $\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d}W^{(1)}}$ should be thought as a gradient of the cost function $L\left(\mathbf{W}\right)$ with respect to the arguments $W^{(1)}_{i,j}$ that is written as a matrix of size $\left(d+1\right)\times d$. In what follows, we denote a matrix $M$ with the first row removed as $\widehat{M}$ and also write $\mathrm{OHV}$ for a one hot vector representation of input data.
$$
\boxed{
\begin{array}{rcl}
\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d} Z^{(\ell)}}&=& 
\begin{cases}
\mathrm{softmax}\left(Z^{(L)}\right) - 
\mathrm{OHV},& \ell = L,\\
X^{(\ell)}\odot\left[
\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d} X^{(\ell)}}
- \Sigma\left(X^{(\ell)}\odot
\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d} X^{(\ell)}} \right)
\right]
,& \ell < L
\end{cases}
\\
\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d} X^{(\ell)}} &=& \frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d} Z^{(\ell+1)}}\cdot \left(\widehat{W}^{(l+1)}\right)^{\top}
\\
\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d} W^{(\ell)}} &=& 
\left(\widetilde{X}^{(\ell-1)}\right)^{\top}\cdot
\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d} Z^{(\ell)}}
\end{array}
},
$$
where for any matrix $M$ we define $\Sigma\left(M\right)$ as a matrix of the same dimensions as $M$ but with every matrix element swapped with a row-sum of matrix elements of matrix $M$.


In this section we will assume that the weights are now represented by a three dimensional NumPy array, which has a form of a row vector, where each element is a two dimensional NumPy array. I.e., weights have a form of

$$
\mathbf{W} = \left[\mathbf{W}^{(1)}, \mathbf{W}^{(2)}, \ldots, \mathbf{W}^{(L)}\right],
$$
and $\mathit{weights}[i,j,k] = \mathbf{W}^{(i)}_{j,k}$. 

The neural network state is now described by $2\cdot L + 1$ matrices. This is given by
$$
\mathit{state} = \left[\mathbf{X}^{(0)}, \mathbf{Z}^{(1)}, \mathbf{X}^{(1)}, \mathbf{Z}^{(2)}, \mathbf{X}^{(2)}, \ldots, \mathbf{Z}^{(L)}, \mathbf{X}^{(L)}\right].
$$


Ideally, the code you will write here should work for any number of layers, despite your task being only to model $2$ layered network. Let us introduce a variable that stores the number of layers, which you can later change to see how this will affect the output.

In [52]:
no_of_layers = 2

1. Start by writing the $\mathtt{nn\_forward\_propagation}$ function that takes two arguments
- _data_input_ - input data for the first layer ($\mathbf{X}^{(0)}$);
- _weight_matrix_ - is the list of weight matrices as described above.

The function should return you the state of neural network following the above formulas. The output should have a form
$$
\left[\mathbf{X}^{(0)}, \mathbf{Z}^{(1)}, \mathbf{X}^{(1)}, \mathbf{Z}^{(2)}, \mathbf{X}^{(2)}, \ldots, \mathbf{Z}^{(L)}, \mathbf{X}^{(L)}\right].
$$

In [53]:
def nn_forward_propagation(data_input, weight_matrix):    
    no_of_layers = weight_matrix.shape[0]
    nn_state = np.empty(2*no_of_layers+1, dtype = np.ndarray)
    nn_state[0] = data_input
    for l in range(no_of_layers):
        nn_state[2*l+1] = model_function(linear_regression_data(nn_state[2*l]),weight_matrix[l])
        nn_state[2*l+2] = softmax_function(nn_state[2*l+1], axis = 1)
    return nn_state

Test your function with the following unit test

In [46]:
test_data_input = np.array([[1,2],[3,4],[5,6]])
test_weight_matrix = np.array([[[1,2],[3,4],[5,6]], [[-1,2],[-2,3],[3,1]]])
assert_array_almost_equal(nn_forward_propagation(test_data_input,test_weight_matrix)[0],np.array([[1,2],[3,4],[5,6]]))
assert_array_almost_equal(nn_forward_propagation(test_data_input,test_weight_matrix)[1],np.array([[14,18],[30,38],[46,58]]))
assert_array_almost_equal(nn_forward_propagation(test_data_input,test_weight_matrix)[2],\
                          np.array([[1.798621e-02, 9.820138e-01],[3.353501e-04, 9.996646e-01],[6.144175e-06, 9.999939e-01]]))
assert_array_almost_equal(nn_forward_propagation(test_data_input,test_weight_matrix)[3],\
                          np.array([[1.910069, 3.035972],[1.998323, 3.000671],[1.999969, 3.000012]]))
assert_array_almost_equal(nn_forward_propagation(test_data_input,test_weight_matrix)[4],\
                          np.array([[0.244918, 0.755082], [0.26848 , 0.73152 ], [0.268933, 0.731067]]))

2. Implement the function $\mathtt{nn\_back\_propagation}$ that takes $3$ arguments:
- _data_input_ - input data for the first layer ($\mathbf{X}^{(0)}$);
- _weight_matrix_ - is the list of weight matrices as described above;
- _one_hot_vector_encoding_ - one hot vector encoding of training data samples.

Your function should output a gradient $\frac{\mathrm{d}L\left(\mathbf{W}\right)}{\mathrm{d}\mathbf{W}}$ as per the above formulas. it should have the same shape as _weight_matrix_. You should first evaluate a state of the neural network by following forward propagation method, and then evaluate corresponding derivatives by using the above formulas.

In [47]:
def nn_back_propagation(data_input, weight_matrix, one_hot_vector_encodings):    
    nn_state = nn_forward_propagation(data_input, weight_matrix)
    no_of_layers = weight_matrix.shape[0]
    full_X_gradient = np.empty(no_of_layers, dtype = np.ndarray)
    full_W_gradient = np.empty(no_of_layers, dtype = np.ndarray)
    full_Z_gradient = np.empty(no_of_layers, dtype = np.ndarray)
    
    full_Z_gradient[no_of_layers-1] = nn_state[-1] - one_hot_vector_encodings
    for layer in range(no_of_layers,0,-1):
        full_W_gradient[layer-1] = linear_regression_data(nn_state[2*layer-2]).T@full_Z_gradient[layer-1]
        full_X_gradient[layer-1] = full_Z_gradient[layer-1] @ (np.delete(weight_matrix[layer-1],(0), axis = 0)).T
        if layer != 1:
            product_matrix = full_X_gradient[layer-1]*nn_state[2*layer-2]
        sigma_matrix = np.sum(product_matrix,axis=1)
        full_Z_gradient[layer-2] = product_matrix - (nn_state[2*layer-2].T*sigma_matrix).T
        
    return full_W_gradient

Test your function with the following unit test.

In [48]:
test_data_input = np.array([[1,2],[3,4],[5,6]])
test_weight_matrix = np.array([[[1,2],[3,4],[5,6]], [[-1,2],[-2,3],[3,1]]])
assert_array_almost_equal(nn_back_propagation(test_data_input, test_weight_matrix, np.array([[0,1],[1,0],[0,1]]))[0],\
                         np.array([[-0.028576,  0.028576], [-0.025189,  0.025189], [-0.053766,  0.053766]]))
assert_array_almost_equal(nn_back_propagation(test_data_input, test_weight_matrix, np.array([[0,1],[1,0],[0,1]]))[1],\
                         np.array([[-0.217669,  0.217669], [ 0.004161, -0.004161], [-0.22183 ,  0.22183 ]]))

3. Write two functions 
- $\mathtt{nn\_cost\_function}$ which evaluates a cost function by following the above formula. This function should take $3$ arguments: _data_input_, _weight_matrix_, _one_hot_vector_encodings_ and return a value of cost function. 
- $\mathtt{nn\_prediction\_function}$ which evaluates class labels for data samples. This function should take $2$ arguments: _data_input_, _weight_matrix_ and return a vector of class labels.

In [49]:
def nn_cost_function(data_input, weight_matrix, one_hot_vector_encodings):    
    nn_state = nn_forward_propagation(data_input, weight_matrix)
    return np.sum(np.log(np.sum(np.exp(nn_state[-2]), axis=1))- np.sum(one_hot_vector_encodings*nn_state[-2], axis=1))

In [50]:
def nn_prediction_function(data_input, weight_matrix):    
    nn_state = nn_forward_propagation(data_input, weight_matrix)
    return np.argmax(nn_state[-1], axis=1)

4. Finally, we would like to apply all the above to data. I.e.,
- define two lambda functions called $\mathtt{nn\_objective}$, $\mathtt{nn\_gradient}$ that take $1$ argument _weight_;
- define nn_initial_weights_ to be a vector of zero matrices of appropriate sizes;
- let the step size parameter _nn_step_size_ be equal to $3.9/\left\|\mathbf{X}\right\|^2$;
- run 10,000 iterations of the gradient descent procedure to find an optimal weights nn_optimal_weights_;
- evaluate an accuracy rate nn_accuracy_rate_ evaluated for the model defined to have nn_optimal_weights_ weights.


In [55]:
complete_data = np.genfromtxt("dataset.csv", skip_header = 1,delimiter = ',')
labels = complete_data[:, -1].astype(int) - np.min(complete_data[:, -1].astype(int))
inputs = complete_data[:, :-1]

no_of_layers = 2
data_inputs, row_of_means, row_of_stds = standardise(inputs)
data_matrix = linear_regression_data(data_inputs)

OHV = one_hot_vector_encoding(labels)


nn_objective = lambda weight_matrix: nn_cost_function(data_inputs,weight_matrix,OHV)
nn_gradient = lambda weight_matrix: nn_back_propagation(data_inputs,weight_matrix,OHV)
nn_step = 3.9/(np.linalg.norm(data_matrix, 2) ** 2)
nn_initial_weight_matrix = np.empty(no_of_layers, dtype = np.ndarray)


for weight_pos in range(no_of_layers - 1):
    nn_initial_weight_matrix[weight_pos] = np.zeros((data_inputs.shape[1]+1,data_inputs.shape[1]))

nn_initial_weight_matrix[no_of_layers - 1] = np.zeros((data_inputs.shape[1]+1,OHV.shape[1]))
nn_optimal_weights, _ = gradient_descent(nn_objective,nn_gradient, 
                                              nn_initial_weight_matrix,step_size=nn_step,
                                              no_of_iterations=10000, print_output=1000)

nn_accuracy_rate = classification_accuracy(nn_prediction_function(data_inputs, nn_optimal_weights),labels,)
print("The two layered neural network successfully classified {acc:2.2f} % of data".format(acc = 100 * nn_accuracy_rate))

Iteration 1000/10000, objective = 1856.2330085332792.
Iteration 2000/10000, objective = 1520.4452454103941.
Iteration 3000/10000, objective = 1499.9629638478948.
Iteration 4000/10000, objective = 1494.5719839263595.
Iteration 5000/10000, objective = 1491.7019577532692.
Iteration 6000/10000, objective = 1489.8868257828972.
Iteration 7000/10000, objective = 1488.6348941628878.
Iteration 8000/10000, objective = 1487.7227615588802.
Iteration 9000/10000, objective = 1487.0321824162013.
Iteration 10000/10000, objective = 1486.4941591560323.
Iteration completed after 10000/10000, objective = 1486.4941591560323.
The two layered neural network successfully classified 61.14 % of data
